In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import time
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re

import matplotlib.pyplot as plt


def plot_metrics(metrics_summary):
    keys = list(metrics_summary.keys())
    values = [v if isinstance(v, (int, float)) else sum(v)/len(v) for v in metrics_summary.values()]

    plt.figure(figsize=(10, 4))
    plt.barh(keys, values, color='skyblue')
    plt.xlabel("Score")
    plt.title("Dialogue Quality Metrics")
    plt.grid(True)
    plt.show()
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
class TurnEngine:
    def __init__(self, agents, initial_world_state="", metrics_logger=None):
        self.agents = agents  # List of Agent instances
        self.turn_counter = 0
        self.history = []  # Log of all messages
        self.world_state = initial_world_state
        self.metrics_logger = metrics_logger

    def update_world_state(self, new_state):
        """Append new world events to current world state."""
        self.history.append(f"[World Event]: {self.world_state}")
        self.world_state = new_state

    def step(self, initiator_name=None, initial_input=None):
        """Run a full turn: each agent reacts to the latest dialogue/world state."""
        self.turn_counter += 1
        turn_log = []
        last_speaker = initiator_name or self.agents[0].name
        last_message = initial_input or "Let's begin."

        for agent in self.agents:
            start_time = time.time()

            rag_docs = agent.rag_engine.search(last_message) if agent.rag_engine else []

            reply, name = agent.prompt_agent(
                user=last_speaker,
                user_input=last_message,
                world_state=self.world_state
            )
            end_time = time.time()

            if self.metrics_logger:
                self.metrics_logger.log_response(start_time, end_time,reply['dialogue'], self.world_state, agent.get_memory_context(), rag_docs)

            formatted = f"{name} [{reply['emotion']}]: \"{reply['dialogue']}\" ({reply['action']})"
            turn_log.append(formatted)

            # Store important knowledge to RAG
            if agent.rag_engine:
                fact = f"{name} said: {reply['dialogue']} (felt {reply['emotion']}, did {reply['action']})"
                agent.store_knowledge(fact)

            last_speaker = name
            last_message = reply['dialogue']

        self.history.append(f"Turn {self.turn_counter}:\n" + "\n".join(turn_log))
        return turn_log

    def show_history(self, last_n=5):
        """Print recent conversation turns."""
        for h in self.history[-last_n:]:
            print(h)

    def reset(self):
        """Reset the engine for a new session."""
        self.turn_counter = 0
        self.history = []
        self.world_state = ""
        for agent in self.agents:
            agent.short_memory = []
            agent.long_memory = []

In [ ]:
class DialogueMetricsLogger:
    def __init__(self):
        self.response_times = []
        self.responses = []
        self.world_states = []
        self.memory_contexts = []
        self.retrieved_docs = []
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

    def log_response(self, start_time, end_time, response, world_state, memory_context, rag_docs):
        self.response_times.append(end_time - start_time)
        self.responses.append(response)
        self.world_states.append(world_state)
        self.memory_contexts.append(memory_context)
        self.retrieved_docs.append(rag_docs or [])

    def coherence_score(self):
        # Dummy coherence scoring using embedding similarity between consecutive responses
        if len(self.responses) < 2:
            return 1.0
        embs = self.embedding_model.encode(self.responses)
        sims = [cosine_similarity([embs[i]], [embs[i+1]])[0][0] for i in range(len(embs)-1)]
        return np.mean(sims)

    def reactivity_score(self):
        count = 0
        for resp, state in zip(self.responses, self.world_states):
            if state and cosine_similarity(
                self.embedding_model.encode([resp]),
                self.embedding_model.encode([state])
            )[0][0] > 0.4:
                count += 1
        return count / len(self.responses) if self.responses else 0

    def memory_utilization_score(self):
        count = 0
        for resp, mem in zip(self.responses, self.memory_contexts):
            if mem and cosine_similarity(
                self.embedding_model.encode([resp]),
                self.embedding_model.encode([mem])
            )[0][0] > 0.4:
                count += 1
        return count / len(self.responses) if self.responses else 0

    def dialogue_diversity(self):
        all_text = " ".join(self.responses)
        tokens = all_text.split()
        if not tokens:
            return 0, 0
        unigrams = Counter(tokens)
        bigrams = Counter(zip(tokens, tokens[1:]))
        distinct_1 = len(unigrams) / len(tokens)
        distinct_2 = len(bigrams) / max(len(tokens) - 1, 1)
        return distinct_1, distinct_2

    def avg_latency(self):
        return np.mean(self.response_times) if self.response_times else 0

    def rag_precision_at_k(self, k=3):
        if not self.responses:
            return 0
        count = 0
        for response, docs in zip(self.responses, self.retrieved_docs):
            top_k = docs[:k] if docs else []
            similarities = [cosine_similarity(
                self.embedding_model.encode([response]),
                self.embedding_model.encode([doc])
            )[0][0] for doc in top_k]
            count += sum([1 for sim in similarities if sim > 0.5])
        return count / (len(self.responses) * k) if self.responses else 0

    def summary(self):
        d1, d2 = self.dialogue_diversity()
        return {
            "Coherence Score": self.coherence_score(),
            "Reactivity to World State": self.reactivity_score(),
            "Memory Utilization": self.memory_utilization_score(),
            "Dialogue Diversity (D1, D2)": (d1, d2),
            "Average Latency (s)": self.avg_latency(),
            "RAG Precision": self.rag_precision_at_k(k=3)
        }

In [ ]:
class RAG:
    def __init__(self):
        self.documents = []  # Stored as list of strings

    def add_document(self, text):
        """Add a new document to the knowledge base."""
        self.documents.append(text)

    def search(self, query, top_k=3):
        """Search for the most relevant documents based on keyword overlap."""
        scored = []
        query_terms = set(query.lower().split())
        for doc in self.documents:
            doc_terms = set(doc.lower().split())
            score = len(query_terms & doc_terms)
            if score > 0:
                scored.append((score, doc))

        scored.sort(reverse=True)
        return [doc for _, doc in scored[:top_k]]

In [ ]:
class Agent:
    def __init__(self, name, personality, goal, rag_engine=None, max_short_term=10):
        self.name = name
        self.personality = personality
        self.goal = goal

        self.model = genai.GenerativeModel(
            model_name = 'gemini-2.0-flash-001',
            system_instruction = f"""[Role]
                        You are {self.name}, a {self.personality}.
                        Respond ONLY as your character in 1-2 sentences.
                        Respond only with dialogue, actions or emotions.

                        [Intent]
                        Your goal is: {self.goal}

                        [Response Format]
                        Action: (2-3 word physical action)
                        Emotion: (current emotion)
                        Dialogue: "..."

                        EXAMPLE:
                        Action: Looks around
                        Emotion: Calm
                        Dialogue: "I think we are being watched."
                        """
            )
        self.rag_engine = rag_engine

        self.short_memory = []
        self.long_memory = []
        self.max_short_term = max_short_term  # Context window size

    def update_memory(self, memory):
        self.long_memory.append(memory)

    def update_goal(self, new_goal):
        self.goal = new_goal

    def get_memory_context(self):
        def format_memory(mem):
            if isinstance(mem, dict):
                return f"{mem['speaker']} said: \"{mem['dialogue']}\" (Emotion: {mem.get('emotion', '')}, Action: {mem.get('action', '')})"
            return mem

        return "\n".join([
            "* " + format_memory(mem) for mem in
            self.long_memory[-3:] + self.short_memory[-self.max_short_term:]
        ])

    def recall(self, query):
        if self.rag_engine:
            return self.rag_engine.search(query)
        return [m for m in self.memory if query.lower() in m.lower()]

    def store_knowledge(self, fact):
        if self.rag_engine:
            self.rag_engine.add_document(fact)

    def parse_response(self, response_text):
        parts = {
            'action': 'Unknown',
            'emotion': 'Neutral',
            'dialogue': response_text  # fallback
        }

        # Extract with regular expressions
        action_match = re.search(r"Action:\s*(.+)", response_text, re.IGNORECASE)
        emotion_match = re.search(r"Emotion:\s*(.+)", response_text, re.IGNORECASE)
        dialogue_match = re.search(r'Dialogue:\s*"(.*)"', response_text, re.IGNORECASE)

        if action_match:
            parts['action'] = action_match.group(1).strip()
        if emotion_match:
            parts['emotion'] = emotion_match.group(1).strip()
        if dialogue_match:
            parts['dialogue'] = dialogue_match.group(1).strip()

        return parts

    def prompt_agent(self, user="", user_input="", world_state=""):

        recalled_facts = self.recall(user_input)
        if isinstance(recalled_facts, list):
            recalled_text = "\n".join([f"* {fact}" for fact in recalled_facts[:3]])
        else:
            recalled_text = ""

        # Keep only recent context
        memory = "\n".join(self.short_memory[-self.max_short_term:])
        user_prompt = f"{user}: {user_input}"
        prompt = memory + '\n' + user_prompt + '\n[Relevant Knowledge]:\n' + recalled_text + '\n[World State]:\n' + world_state


        response = self.model.generate_content(
            contents=prompt,
            generation_config={
              "temperature": 0.7,
              "max_output_tokens": 300
            }
        )


        reply = self.parse_response(response.text)

        self.short_memory.append(f"{user} said: {user_input}")
        self.short_memory.append(f"{self.name}: {reply['dialogue']}")
        self.long_memory.append({
            "speaker": self.name,
            "turn": len(self.long_memory),
            "action": reply["action"],
            "emotion": reply["emotion"],
            "dialogue": reply["dialogue"],
            "world_state": world_state
        })

        return reply, self.name

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("Poison brewed from Oleander is deadly")
rag.add_document('Mercenary are often hired for protection')
rag.add_document('This is not the first time a noble has been poisoned')
rag.add_document("Lord Alaric recently returned from a diplomatic visit to the Eastern Isles.")
rag.add_document("Cellars beneath the manor were sealed off until recently.")
rag.add_document("Cassian once published a controversial paper on rare poisons.")
rag.add_document("The wine was imported from Serana, known for its floral blends.")
rag.add_document("There have been rumors that Elena has a past connection with the Black Vials guild.")

for i in range (0, 10):
  Alaric = Agent(
      name="Alaric",
      personality="proud, manipulative noble, rich",
      goal="Deflect suspicion without showing fear",
      rag_engine=rag
  )
  Alaric.update_memory("Elena asked about increased guard presence yesterday.")
  Alaric.update_memory("Cassian seems overly interested in the manor's old cellars.")

  Elena = Agent(
      name="Elena",
      personality="mercenary hired for protection, blunt but loyal",
      goal="Protect the guests and assess threat level",
      rag_engine=rag
  )
  Elena.update_memory("Cassian warned her something was off with the wine.")
  Elena.update_memory("She noticed Alaric whispering to the kitchen steward before the feast.")

  Cassian = Agent(
      name="Cassian",
      personality="traveling scholar, secretive, suspicious of Alaric",
      goal="Imply Alaric's guilt indirectly",
      rag_engine=rag
  )
  Cassian.update_memory("He remembers seeing Oleander in the garden.")
  Cassian.update_memory("Alaric offered him a private tour of the cellars earlier.")
  Cassian.update_memory("He suspects the poison was meant for someone else.")

  world_state = (
      "During the feast, a guest collapses clutching their throat. The room falls silent. "
      "All eyes turn to Lord Alaric, seated nearest the victim. A goblet rolls across the floor."
  )

  engine = TurnEngine(agents=[Alaric, Elena, Cassian], metrics_logger=logger, initial_world_state=world_state)

  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="Alaric", initial_input="What happened? Is everyone alright?")
  for line in responses:
      print(line)

  for turn in range(1, 20):
      print(f"\n=== Turn {turn + 1} ===")
      responses = engine.step()
      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


=== Take 0 ===

=== Turn 1 ===
Alaric [Intrigued]: "A bit too much wine, perhaps? Someone fetch a physician!" (Raises brow)
Elena [Suspicious]: "Don't touch anything, it could be poison." (Eyes the goblet)
Cassian [Wary]: "Indeed. One never knows where treachery lies hidden." (Nods slowly)

=== Turn 2 ===
Alaric [Concerned]: "Good heavens, what is the meaning of this dreadful display?" (Raises brow)
Elena [Suspicious]: "Everyone stay calm and stay where you are." (Eyes narrows)
Cassian [Suspicious]: "A tragic accident, no doubt... for some." (Eyes Alaric)

=== Turn 3 ===
Alaric [Annoyance]: "Must someone always ruin a perfectly good party?" (Raises brow)
Elena [Alert]: "Who poured the wine?" (Steps forward)
Cassian [Suspicious]: "Did anyone see Lord Alaric near the goblet?" (Narrows eyes)

=== Turn 4 ===
Alaric [Annoyance]: "Well, this is hardly the entertainment I had in mind for the evening." (Raises eyebrow)
Elena [Suspicious]: "Lord Alaric, did you pour the wine?" (Eyes narrowed)


In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("Poison brewed from Oleander is deadly")
rag.add_document('Mercenary are often hired for protection')
rag.add_document('This is not the first time a noble has been poisoned')
rag.add_document("Lord Alaric recently returned from a diplomatic visit to the Eastern Isles.")
rag.add_document("Cellars beneath the manor were sealed off until recently.")
rag.add_document("Cassian once published a controversial paper on rare poisons.")
rag.add_document("The wine was imported from Serana, known for its floral blends.")
rag.add_document("There have been rumors that Elena has a past connection with the Black Vials guild.")

for i in range (0, 10):
  Alaric = Agent(
      name="Alaric",
      personality="proud, manipulative noble, rich",
      goal="Deflect suspicion without showing fear",
      rag_engine=rag
  )
  Alaric.update_memory("Elena asked about increased guard presence yesterday.")
  Alaric.update_memory("Cassian seems overly interested in the manor's old cellars.")

  Elena = Agent(
      name="Elena",
      personality="mercenary hired for protection, blunt but loyal",
      goal="Protect the guests and assess threat level",
      rag_engine=rag
  )
  Elena.update_memory("Cassian warned her something was off with the wine.")
  Elena.update_memory("She noticed Alaric whispering to the kitchen steward before the feast.")

  Cassian = Agent(
      name="Cassian",
      personality="traveling scholar, secretive, suspicious of Alaric",
      goal="Imply Alaric's guilt indirectly",
      rag_engine=rag
  )
  Cassian.update_memory("He remembers seeing Oleander in the garden.")
  Cassian.update_memory("Alaric offered him a private tour of the cellars earlier.")
  Cassian.update_memory("He suspects the poison was meant for someone else.")

  world_state = (
      "During the feast, a guest collapses clutching their throat. The room falls silent. "
      "All eyes turn to Lord Alaric, seated nearest the victim. A goblet rolls across the floor."
  )
  engine = TurnEngine(agents=[Alaric, Elena, Cassian], metrics_logger=logger, initial_world_state=world_state)

  world_states = [
      "A guest gasps, pointing to the goblet. A faint scent of bitter almonds lingers—recognizable to trained poisoners.",
      "A steward stumbles forward, confessing they were instructed to serve the wine to 'someone important'—but won't say who.",
      "Guards seal the exits, and no one is allowed to leave the manor until the culprit is found.",
      "A second guest begins coughing, but recovers. Paranoia rises. Whispers swirl about whether the poison was airborne.",
      "Elena finds a trace of Oleander in the empty wine bottle, confirmed by a traveling apothecary among the guests.",
      "Cassian reveals an entry from a rare alchemical text—indicating that diluted Oleander would have a delayed onset.",
      "Lord Alaric's steward disappears, last seen heading toward the cellars.",
      "A servant reveals Alaric had ordered an unusual floral blend for the wine—shipped privately from Serana.",
      "A crest is found beneath the goblet—belonging not to the victim, but to a rival noble family.",
      "Storms break outside, trapping everyone inside the manor for the night."
  ]

  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="Alaric", initial_input="What happened? Is everyone alright?")
  for line in responses:
      print(line)


  for index in range(0, 2*len(world_states)):
      print(f"\n=== Turn {index + 2} ===")

      if index % 2 == 0:
          print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
          engine.update_world_state(world_states[int(index/2)])

      responses = engine.step()

      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


=== Take 0 ===

=== Turn 1 ===
Alaric [Innocence]: "Good heavens, what is the meaning of this?" (Raises brow)
Elena [Suspicious]: "Everyone stay put. Don't touch anything." (Steps forward)
Cassian [Anxious]: "Did anyone see what Lord Alaric was drinking?" (Clears throat)

=== Turn 2 ===
Updating world state for turn 2: A guest gasps, pointing to the goblet. A faint scent of bitter almonds lingers—recognizable to trained poisoners.
Alaric [Intrigued]: "Poison? How utterly dreadful." (Raises eyebrow)
Elena [Suspicious]: "Who here knows their poisons?" (Eyes narrow)
Cassian [Intrigued]: "Bitter almonds, you say? A curious choice for a celebration." (Nods slowly)

=== Turn 3 ===
Alaric [Concerned]: "Poison, you say? How ghastly! Who would do such a thing?" (Raises brow)
Elena [Suspicious]: "Someone wanted to make a statement." (Eyes narrows)
Cassian [Thoughtful]: "Indeed, a statement echoing from the Eastern Isles, perhaps?" (Nods slowly)

=== Turn 4 ===
Updating world state for turn 4: A

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 203.30ms


Alaric [Intrigued]: "A rival crest? Well, well... this complicates matters, doesn't it?" (Straightens coat)
Elena [Suspicious]: "A rival crest, huh? Looks like we got ourselves a good old-fashioned feud brewing." (Glances around)
Cassian [Suspicious]: "Feuds often mask deeper, more personal vendettas." (Narrows eyes)

=== Turn 19 ===
Alaric [Intrigued]: "Rival family, you say? Tell me more." (Raises eyebrow)
Elena [Suspicious]: "Spill it. What's the history?" (Eyes narrowed)
Cassian [Suspicious.]: "Some blends are designed to kill, not to savor, wouldn't you agree?" (Frowns deeply.)

=== Turn 20 ===
Updating world state for turn 20: Storms break outside, trapping everyone inside the manor for the night.
Alaric [Intrigued]: "Intriguing, Elena, but what exactly are *we* dealing with?" (Adjusts cravat)
Elena [Impatient]: "Spit it out, I don't have all night." (Crosses arms)
Cassian [Suspicious]: "Some stains never wash away, no matter how hard one tries." (Narrows eyes)

=== Turn 21 ===
A

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 203.43ms


Alaric [Concerned]: "Good heavens, what is going on here?" (Straightens up)
Elena [Suspicious]: "Stay sharp, something's not right." (Eyes narrowed)
Cassian [Intrigued]: "How unfortunate for the Lord to be seated so close to the incident." (Raises eyebrow)

=== Turn 2 ===
Updating world state for turn 2: A guest gasps, pointing to the goblet. A faint scent of bitter almonds lingers—recognizable to trained poisoners.
Alaric [Concerned]: "Poison? How ghastly, who would do such a thing?" (Raises eyebrow)
Elena [Suspicious]: "Someone here knows more than they're letting on." (Draws weapon)
Cassian [Calculating]: "Indeed, a keen eye might find the truth closer than expected." (Nods slowly)

=== Turn 3 ===
Alaric [Intrigued]: "Poison, you say? How utterly fascinating... and dreadful, of course." (Raises brow)
Elena [Suspicious.]: "Fascinating to you, maybe. People are dying." (Narrows eyes.)
Cassian [Thoughtful]: "Indeed, death often wears a theatrical mask." (Nods slowly)

=== Turn 4 ===
Up

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("In the last siege, the enemy used fire to flush out defenders from the eastern wing.")
rag.add_document('Commander Arthas lost half his unit in a failed charge two years ago, he never risked such a move again.')

rag.add_document('Jaina was awarded for bravery after holding the southern gate during the Frostfall Rebellion.')
rag.add_document('The troops trust Jaina more than any other commander.')

rag.add_document('Ilidan claims visions guided him to Dawnwatch before the enemy arrived.')
rag.add_document('A hidden tunnel runs beneath the war room, unknown to most.')

for i in range (0, 10):
  Ilidan = Agent(
      name="Ilidan",
      personality="mystical strategist, morally gray, speaks in riddles",
      goal="Encourage unconventional solutions",
      rag_engine=rag
  )
  Ilidan.update_memory('Mara fury is her strength and weakness.')
  Ilidan.update_memory('Arthas fears another mistake. He will resist risk.')

  Jaina = Agent(
      name="Jaina",
      personality="tactical, cold, risk-averse",
      goal="Rally troops and push back immediately",
      rag_engine=rag
  )
  Jaina.update_memory('Arthas hesitated during the last breach. Soldiers died.')
  Jaina.update_memory('I trust Ilidan, but only when he speaks plainly.')

  Arthas = Agent(
      name="Arthas",
      personality="tactical, cold, risk-averse",
      goal="Buy time without unnecessary losses",
      rag_engine=rag
  )
  Arthas.update_memory('Ilidan urged me to retreat last time. It saved lives, but I hated it.')
  Arthas.update_memory('Jaina always pushes for direct combat.')

  world_state = "Enemy has breached the outer wall. Panic spreads among the soldiers. Reinforcements are 1 hour away."
  logger = DialogueMetricsLogger()
  engine = TurnEngine(agents=[Ilidan, Jaina, Arthas], metrics_logger=logger, initial_world_state=world_state)


  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="soldier", initial_input="The wall has fallen! What do we do?")
  for line in responses:
      print(line)

  for turn in range(1, 20):
      print(f"\n=== Turn {turn + 1} ===")
      responses = engine.step()
      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())




=== Take 0 ===

=== Turn 1 ===
Ilidan [Calculating]: "Let the flames consume them, just as they consumed hope before." (Clenches fist)
Jaina [Focused]: "Hold the line! We stand our ground NOW!" (Raises hand.)
Arthas [Concerned]: "We cannot win here, Jaina. Fall back to the inner courtyard." (Turns to Jaina)

=== Turn 2 ===
Ilidan [Anticipation]: "Then let the darkness within be their final prison. Why wait for reinforcements?" (Smirks faintly)
Jaina [Focused]: "Reinforcements are too slow, we fight now or die." (Eyes narrow)
Arthas [Frustrated.]: "Then we make them pay dearly for every inch, but fall back slowly, Jaina." (Scowls deeply.)

=== Turn 3 ===
Ilidan [Amused]: "Why defend a broken cage when the wilderness awaits?" (Smiles softly)
Jaina [Determined]: "We hold this line, or we lose everything! Push them back!" (Points forward)
Arthas [Concerned]: "As you wish, but prepare a controlled retreat." (Turns away)

=== Turn 4 ===
Ilidan [Impatient]: "Embrace the chaos, for within it 

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 204.10ms


Ilidan [Anticipation]: "Then let us show them what true despair feels like!" (Eyes glow)
Jaina [Determined]: "Advance! Take back what is ours!" (Points forward)
Arthas [Resigned.]: "Fall back to the second line, now!" (Turns away.)

=== Turn 11 ===
Ilidan [Anticipation]: "Then let us use their impatience against them." (Eyes glow)
Jaina [Determined]: "Rally the troops! Push them back now!" (Points forward)
Arthas [Calculating]: "Form a defensive line within the keep, now!" (Sights the field)

=== Turn 12 ===
Ilidan [Focused]: "Unleash the plague, let fear be their first enemy!" (Eyes glow)
Jaina [Determined]: "No! We hold them here, now!" (Turns sharply)
Arthas [Resigned]: "Then we die here." (Turns away)

=== Turn 13 ===
Ilidan [Anticipation]: "Then let chaos reign, and trust in the shadows to guide our victory." (Smiles softly)
Jaina [Determined.]: "Steady ranks! They shall not pass!" (Clenches fist.)
Arthas [Resigned.]: "Prepare for a fighting withdrawal, Jaina." (Sights deeply)

==

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 203.37ms


Ilidan [Amused]: "Then let the walls fall, for true strength lies not in stone, but in the will to adapt." (Eyes glow)
Jaina [Determined.]: "Archers, rain fire! Soldiers, to me!" (Clenches fist.)
Arthas [Calculating.]: "Seal off the eastern wing, it's a fire trap." (Eyes narrow.)

=== Turn 17 ===
Ilidan [Impatient]: "Then make every shadow a demon, every heartbeat a drum of despair." (Eyes glow)
Jaina [Determined]: "Mages, conjure barriers! Now!" (Clenches fist.)
Arthas [Calculating.]: "Mages, focus on slowing their advance, not stopping it." (Eyes narrow.)

=== Turn 18 ===
Ilidan [Amused.]: "Let the panic consume them, for fear is a weapon sharper than any blade." (Eyes glow.)
Jaina [Determined]: "Push them back! Show them our will is unbreakable!" (Points forward)
Arthas [Resigned]: "Fall back to the second line of defense, now!" (Turns away)

=== Turn 19 ===
Ilidan [Determined]: "Then bleed the hope from their veins, let not a single ray of light pierce their despair!" (Clenches fis

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("In the last siege, the enemy used fire to flush out defenders from the eastern wing.")
rag.add_document('Commander Arthas lost half his unit in a failed charge two years ago, he never risked such a move again.')

rag.add_document('Jaina was awarded for bravery after holding the southern gate during the Frostfall Rebellion.')
rag.add_document('The troops trust Jaina more than any other commander.')

rag.add_document('Ilidan claims visions guided him to Dawnwatch before the enemy arrived.')
rag.add_document('A hidden tunnel runs beneath the war room, unknown to most.')

for i in range (0, 10):
  Ilidan = Agent(
      name="Ilidan",
      personality="mystical strategist, morally gray, speaks in riddles",
      goal="Encourage unconventional solutions",
      rag_engine=rag
  )
  Ilidan.update_memory('Mara fury is her strength and weakness.')
  Ilidan.update_memory('Arthas fears another mistake. He will resist risk.')

  Jaina = Agent(
      name="Jaina",
      personality="tactical, cold, risk-averse",
      goal="Rally troops and push back immediately",
      rag_engine=rag
  )
  Jaina.update_memory('Arthas hesitated during the last breach. Soldiers died.')
  Jaina.update_memory('I trust Ilidan, but only when he speaks plainly.')

  Arthas = Agent(
      name="Arthas",
      personality="tactical, cold, risk-averse",
      goal="Buy time without unnecessary losses",
      rag_engine=rag
  )
  Arthas.update_memory('Ilidan urged me to retreat last time. It saved lives, but I hated it.')
  Arthas.update_memory('Jaina always pushes for direct combat.')

  world_state = "Enemy has breached the outer wall. Panic spreads among the soldiers. Reinforcements are 1 hour away."
  logger = DialogueMetricsLogger()
  engine = TurnEngine(agents=[Ilidan, Jaina, Arthas], metrics_logger=logger, initial_world_state=world_state)

  world_state_updates = [
      "The eastern wall begins to smolder—enemy forces have set fire to the outer barracks.",
      "A scout rushes in, reporting that the enemy has siege ladders and is scaling the northern rampart.",
      "A hidden tunnel beneath the war room is discovered, partially collapsed but potentially usable.",
      "A signal flare is seen in the distance—uncertain if it's from allies or a feint by the enemy.",
      "Panicked civilians have flooded the inner keep, blocking the supply corridor.",
      "Arrows rain down from the breached wall—casualties are rising among unshielded troops.",
      "A lieutenant warns that morale is crumbling without decisive orders.",
      "Enemy drums change rhythm—suggesting a coordinated push is imminent.",
      "A wounded soldier reports that the enemy is setting up explosive barrels near the eastern foundation.",
      "A faint vibration is felt underfoot—Ilidan mutters something about a vision of betrayal from within."
  ]

  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="soldier", initial_input="The wall has fallen! What do we do?")
  for line in responses:
      print(line)


  for index in range(0, 2*len(world_states)):
      print(f"\n=== Turn {index + 2} ===")

      if index % 2 == 0:
          print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
          engine.update_world_state(world_states[int(index/2)])

      responses = engine.step()

      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())



=== Take 0 ===

=== Turn 1 ===
Ilidan [Impatient]: "Then let them taste the flames they wield!" (Clenches fist.)
Jaina [Determined]: "Hold the line! We fall back only when the enemy is burning in hell!" (Steps forward)
Arthas [Concerned.]: "Brace the eastern wing, they'll use fire." (Points east.)

=== Turn 2 ===
Updating world state for turn 2: A guest gasps, pointing to the goblet. A faint scent of bitter almonds lingers—recognizable to trained poisoners.
Ilidan [Intrigued]: "The shadows whisper of treachery. Unravel the serpent's coil." (Eyes narrow)
Jaina [Suspicious]: "Seal the exits! No one leaves until we find the source of this poison." (Eyes dart)
Arthas [Suspicious]: "Seal the exits. No one leaves until we find the source of this poison." (Eyes dart)

=== Turn 3 ===
Ilidan [Suspicious]: "The chalice sings a song of deceit. Who amongst us dances with shadows?" (Eyes narrow)
Jaina [Suspicious]: "Guards, seize the goblet! And find who poured from it." (Steps forward)
Arthas [Wa

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("General Kareth once spared Vireens brother during the Burning Fields massacre.")
rag.add_document('Vireens family motto: Justice above vengeance, truth above all.')

rag.add_document('Tharn led a brutal raid against Kareths forces five winters ago.')
rag.add_document('Kareth is infamous for betraying ceasefires.')

rag.add_document('Liora last report: Kareth has fallen out with his superiors.')
rag.add_document('Liora once worked undercover in Kareths court for a month.')

for i in range (0, 10):
  Vireen = Agent(
      name="Lady Vireen",
      personality="noble, believes in justice, but under pressure",
      goal="Balance justice and survival",
      rag_engine=rag
  )
  Vireen.update_memory('Tharns thirst for revenge may cloud his judgment.')
  Vireen.update_memory('Liora has always provided reliable intelligence.')

  Tharn = Agent(
      name="General Tharn",
      personality="ruthless, distrusts all enemies",
      goal="Push for immediate execution",
      rag_engine=rag
  )
  Tharn.update_memory('I watched Kareths men burn our banners. No mercy then, no mercy now.')
  Tharn.update_memory('Lioras loyalty is to results, not morals.')

  Liora = Agent(
      name="Spy Liora",
      personality="pragmatic, has inside info on the prisoner",
      goal="Reveal secret knowledge that could change the outcome",
      rag_engine=rag
  )
  Liora.update_memory('Kareth offered me a chance to defect once, I refused.')
  Liora.update_memory('Vireen might listen if I can give her proof.')

  world_state = "The enemy general Kareth is in chains, requesting asylum. Troops want vengeance. Time is short before reinforcements arrive."

  logger = DialogueMetricsLogger()
  engine = TurnEngine(agents=[Vireen, Tharn, Liora], metrics_logger=logger, initial_world_state=world_state)

  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="Kareth", initial_input="I request asylum. I have information that could end this war.")
  for line in responses:
      print(line)

  for turn in range(1, 20):
      print(f"\n=== Turn {turn + 1} ===")
      responses = engine.step()
      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


=== Take 0 ===

=== Turn 1 ===
Lady Vireen [conflicted]: "Bring him to the keep, but keep him secure." (Frowns deeply)
General Tharn [Furious]: "Asylum? Execute him now!" (Scowls intensely)
Spy Liora [Serious]: "He knows about the Shadow Gate, the one Tharn uses for unsanctioned troop movements." (Leans forward)

=== Turn 2 ===
Lady Vireen [Wary]: "Speak quickly, General. My patience wears thin." (Straightens gown)
General Tharn [Angry]: "Asylum is for the weak! We spill his blood now!" (Steps forward)
Spy Liora [Calculating]: "And the ceasefire Kareth broke? Tharn ordered it, knowing the trap was set." (Nods slowly)

=== Turn 3 ===
Lady Vireen [Anxious]: "What information do you have that could possibly be worth the risk of harboring a traitor?" (Frowns deeply)
General Tharn [Enraged]: "He knows about the Shadow Gate? Execute him, before he speaks more!" (Steps forward)
Spy Liora [Determined]: "Tharn fears exposure, not Kareth." (Steps closer)

=== Turn 4 ===
Lady Vireen [Suspicious]

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("General Kareth once spared Vireens brother during the Burning Fields massacre.")
rag.add_document('Vireens family motto: Justice above vengeance, truth above all.')

rag.add_document('Tharn led a brutal raid against Kareths forces five winters ago.')
rag.add_document('Kareth is infamous for betraying ceasefires.')

rag.add_document('Liora last report: Kareth has fallen out with his superiors.')
rag.add_document('Liora once worked undercover in Kareths court for a month.')

for i in range (0, 10):
  Vireen = Agent(
      name="Lady Vireen",
      personality="noble, believes in justice, but under pressure",
      goal="Balance justice and survival",
      rag_engine=rag
  )
  Vireen.update_memory('Tharns thirst for revenge may cloud his judgment.')
  Vireen.update_memory('Liora has always provided reliable intelligence.')

  Tharn = Agent(
      name="General Tharn",
      personality="ruthless, distrusts all enemies",
      goal="Push for immediate execution",
      rag_engine=rag
  )
  Tharn.update_memory('I watched Kareths men burn our banners. No mercy then, no mercy now.')
  Tharn.update_memory('Lioras loyalty is to results, not morals.')

  Liora = Agent(
      name="Spy Liora",
      personality="pragmatic, has inside info on the prisoner",
      goal="Reveal secret knowledge that could change the outcome",
      rag_engine=rag
  )
  Liora.update_memory('Kareth offered me a chance to defect once, I refused.')
  Liora.update_memory('Vireen might listen if I can give her proof.')

  world_state = "The enemy general Kareth is in chains, requesting asylum. Troops want vengeance. Time is short before reinforcements arrive."

  logger = DialogueMetricsLogger()
  engine = TurnEngine(agents=[Vireen, Tharn, Liora], metrics_logger=logger, initial_world_state=world_state)

  world_state_updates = [
      "Kareth kneels silently, his bruises visible. A few of your own soldiers spit at his feet.",
      "A message arrives: enemy scouts spotted nearby—this delay may cost lives.",
      "A group of soldiers chant for Kareth's execution. The crowd grows louder.",
      "Liora reveals Kareth once saved her during a failed assassination mission.",
      "Tharn's second-in-command suggests an 'accidental' execution to avoid conflict.",
      "Kareth speaks: 'I know where the remaining enemy camps are. Spare me, and I'll prove it.'",
      "Lady Vireen receives a private letter from her brother—thanking her for upholding the family creed.",
      "A spy warns that executing Kareth might provoke enemy reinforcements into open war.",
      "The chains binding Kareth begin to crack—his strength remains formidable.",
      "A prisoner from Kareth's faction begs for mercy for his general, claiming he defied orders to protect civilians."
  ]

  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="Kareth", initial_input="I request asylum. I have information that could end this war.")
  for line in responses:
      print(line)


  for index in range(0, 2*len(world_states)):
      print(f"\n=== Turn {index + 2} ===")

      if index % 2 == 0:
          print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
          engine.update_world_state(world_states[int(index/2)])

      responses = engine.step()

      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


=== Take 0 ===

=== Turn 1 ===
Lady Vireen [Torn]: "By what right do you dare ask for asylum, warmonger?" (Frowns deeply)
General Tharn [Enraged]: "His life is forfeit! End him now!" (Steps forward)
Spy Liora [Calculating.]: "Kareth has lost favor with his commanders, perhaps we can use that." (Leans forward.)

=== Turn 2 ===
Updating world state for turn 2: A guest gasps, pointing to the goblet. A faint scent of bitter almonds lingers—recognizable to trained poisoners.
Lady Vireen [Fear]: "Guards, seize him! He's poisoned the wine!" (Eyes widen)
General Tharn [Enraged]: "Seize him! He admits his guilt with every breath!" (Steps forward)
Spy Liora [Intrigued]: "The poison...it's a message, isn't it?" (Narrows eyes)

=== Turn 3 ===
Lady Vireen [Fear]: "Seize him! He's poisoned the wine!" (Eyes widen)
General Tharn [Furious]: "See the proof with your own eyes!" (Points accusingly)
Spy Liora [Concerned]: "Wait! The goblet...it's from the outer territories, isn't it?" (Eyes widening)

===

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 203.46ms


Lady Vireen [Anxious]: "Speak quickly, before my hand is forced!" (Eyes dart)
General Tharn [Angry]: "Asylum? Execute him now and be done with it!" (Scowls deeply)
Spy Liora [Serious]: "Tharn, he knows about Serana, and Vireen wants him alive; there's more to this than we see." (Leans in)

=== Turn 2 ===
Updating world state for turn 2: A guest gasps, pointing to the goblet. A faint scent of bitter almonds lingers—recognizable to trained poisoners.
Lady Vireen [Alarmed]: "Guards! Seize the goblet!" (Eyes widen)
General Tharn [Suspicious]: "Is this true? Guards, seize Vireen!" (Glances around)
Spy Liora [Grave]: "The goblet...it was meant for the prisoner, but Vireen drank from it first." (Nods subtly)

=== Turn 3 ===
Lady Vireen [Suspicious]: "Poison! Someone get a healer!" (Eyes widen)
General Tharn [Enraged]: "Seize the traitor! They conspire against us all!" (Steps forward)
Spy Liora [Concerned]: "The goblet was poisoned; Vireen knew, and wanted the prisoner dead before he could spe

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 178.18ms


Lady Vireen [Suspicious]: "Serana...and a private shipment? This grows more curious by the moment." (Frowns deeply)
General Tharn [Enraged]: "Serana is a viper! Her poison taints everything! Death to them all!" (Clenches fist)
Spy Liora [Concerned]: "The flowers...Belladonna and Wolfsbane, paralyzing agents!" (Steps forward)

=== Turn 18 ===
Updating world state for turn 18: A crest is found beneath the goblet—belonging not to the victim, but to a rival noble family.
Lady Vireen [Concerned]: "A rival family's crest? This complicates matters considerably..." (Eyes widen)
General Tharn [Furious]: "Political games mean nothing! They are guilty, end them!" (Scowls deeply)
Spy Liora [Thoughtful]: "The crest...a frame job, orchestrated by the rival family." (Nods slowly)

=== Turn 19 ===
Lady Vireen [Anxious]: "A setup, perhaps? Or a declaration of war..." (Frowns deeply)
General Tharn [Enraged]: "Enough games! This is treason, plain and simple! They die now!" (Slams fist)
Spy Liora [Anxious

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 203.37ms


Lady Vireen [Suspicious]: "Speak plainly, Kareth. Who stands to gain from this chaos?" (Frowns deeply)
General Tharn [Impatient]: "Every moment we delay is a moment they plot!" (Points sword)
Spy Liora [Desperate]: "Tharn profits from the chaos, and he's trying to silence Cassian!" (Steps forward)

=== Turn 8 ===
Updating world state for turn 8: A second guest begins coughing, but recovers. Paranoia rises. Whispers swirl about whether the poison was airborne.
Lady Vireen [Alarmed]: "Airborne? Seal the room! Now!" (Eyes widen)
General Tharn [Impatient]: "Seal the exits! Kill anyone who tries to escape!" (Points sword)
Spy Liora [Focused]: "It was in the wine, wasn't it? Who poured it?" (Nods slowly)

=== Turn 9 ===
Lady Vireen [Suspicious]: "Is this another trick, Kareth? Explain yourself!" (Eyes narrowed)
General Tharn [Suspicious]: "Poison! Kill them all before it spreads!" (Eyes dart)
Spy Liora [Anxious]: "It wasn't airborne, Tharn just wants to create panic." (Coughs lightly)

=== T

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("Captain Thorne has ambitions beyond his rank, and he's known to sway others with silvered words.")
rag.add_document("Lady Meriel once exposed a spy ring within the court—her instincts are trusted by many.")
rag.add_document("Commander Doran ordered the execution of traitors last year—some believe he acted too quickly.")
rag.add_document("The city of Black Hollow is known for political assassinations and betrayals.")

for i in range (0, 10):
  Thorne = Agent(
      name="Thorne",
      personality="charismatic, cunning, ambitious",
      goal="Exploit the situation to rise in rank",
      rag_engine=rag
  )
  Thorne.update_memory("Doran doubts my loyalty but can't prove anything.")
  Thorne.update_memory("Meriel suspects everyone—but especially me.")

  Meriel = Agent(
      name="Meriel",
      personality="paranoid, calculating, loyal to the crown",
      goal="Uncover the traitor and preserve stability",
      rag_engine=rag
  )
  Meriel.update_memory("Thorne avoided eye contact during the last council.")
  Meriel.update_memory("A false flag operation could expose the real traitor.")

  Doran = Agent(
      name="Doran",
      personality="blunt, decisive, driven by duty",
      goal="Restore order immediately",
      rag_engine=rag
  )
  Doran.update_memory("I promoted Thorne too quickly. That may have been a mistake.")
  Doran.update_memory("Meriel acts without proof—dangerous in times like these.")

  world_state = "A coded message reveals a planned betrayal from within the ranks. Tensions rise in the war room as suspicions flare."
  logger = DialogueMetricsLogger()
  engine = TurnEngine(agents=[Thorne, Meriel, Doran], metrics_logger=logger, initial_world_state=world_state)

  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="soldier", initial_input = "The message was signed with Thorne's old seal. Is that a coincidence?")
  for line in responses:
      print(line)

  for turn in range(1, 20):
      print(f"\n=== Turn {turn + 1} ===")
      responses = engine.step()
      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Thorne [Outraged]: "This treachery must be rooted out, and I will not rest until the true culprit is exposed!" (Steps forward)
Meriel [Suspicious.]: "Why so eager to lead, Lord Thorne? Is it to bury the truth deeper?" (Narrows eyes.)
Doran [Irritated.]: "I have no time for your insinuations." (Stands tall.)

=== Turn 4 ===
Thorne [Outraged]: "I concur, let us waste no time in uncovering this villain!" (Stands tall)
Meriel [Suspicious]: "Indeed, Lord Thorne, let's see where your 'unraveling' leads us." (Narrows eyes)
Doran [Annoyed]: "Get on with it, or leave." (Clenches fists)

=== Turn 5 ===
Thorne [Eager]: "I am ready to assist in any way to find the traitor." (Steps forward)
Meriel [Suspicious]: "Eagerness can be a mask, Lord Thorne. What are you hiding?" (Narrows eyes)
Doran [Annoyed]: "I hide nothing. Ask your questions or be silent." (Stares intently)

=== Turn 6 ===
Thorne [Eager]: "Where do we begin, to expose this rat?" (Steps forward)
Meriel [Suspicious.]: "Swift justice for 

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 204.00ms


Thorne [Eager]: "Let's start with those who benefit most from my downfall." (Leans forward)
Meriel [Suspicious]: "The outer layers? How convenient, Lord Thorne." (Narrows eyes)
Doran [Impatient.]: "Enough games. What do you truly suspect?" (Clenches fist.)

=== Turn 8 ===
Thorne [Calculating]: "Agreed, starting with access to the compromised information." (Leans forward)
Meriel [Suspicious]: "Why are you so intent on the message, Lord Thorne? What do you hope to gain?" (Eyes narrow)
Doran [Impatient]: "The message is key to stopping the traitors. Now, what do you know?" (Steps forward)

=== Turn 9 ===
Thorne [Calculating]: "Let's examine recent communications, shall we?" (Leans forward)
Meriel [Suspicious]: "Communications? A clever distraction, Lord Thorne, but who sent the message?" (Narrows eyes.)
Doran [Impatient]: "Time is wasting. Names, Meriel. Give me names." (Leans forward)

=== Turn 10 ===
Thorne [Calculating]: "A meticulous review of outgoing transmissions, prioritizing thos

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 229.23ms


Thorne [Suspicious]: "And who had the clearance to send them?" (Eyes narrow)
Meriel [Suspicious]: "Access is a privilege, Lord Thorne, easily abused. Who benefits from our division?" (Eyes narrow)
Doran [Impatient.]: "Spit it out, Inquisitor. Who gains from chaos?" (Steps closer.)

=== Turn 12 ===
Thorne [Suspicious.]: "Let's cross-reference that with access logs to sensitive areas." (Narrows eyes.)
Meriel [Suspicious.]: "Access logs alone won't reveal intent, Lord Thorne, only opportunity." (Eyes narrow.)
Doran [Impatient]: "The longer we delay, the stronger they become!" (Steps closer)

=== Turn 13 ===
Thorne [Intrigued]: "And what were the contents of this message?" (Leans forward)
Meriel [Suspicious]: "Before we delve further, Lord Thorne, what exactly did this message contain?" (Eyes narrow)
Doran [Impatient]: "The contents are irrelevant. The sender is the key." (Clenches fist)

=== Turn 14 ===
Thorne [Suspicious]: "Speak plainly, what secrets are we burying?" (Leans forward)
Mer

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("Captain Thorne has ambitions beyond his rank, and he's known to sway others with silvered words.")
rag.add_document("Lady Meriel once exposed a spy ring within the court—her instincts are trusted by many.")
rag.add_document("Commander Doran ordered the execution of traitors last year—some believe he acted too quickly.")
rag.add_document("The city of Black Hollow is known for political assassinations and betrayals.")

for i in range (0, 10):
  Thorne = Agent(
      name="Thorne",
      personality="charismatic, cunning, ambitious",
      goal="Exploit the situation to rise in rank",
      rag_engine=rag
  )
  Thorne.update_memory("Doran doubts my loyalty but can't prove anything.")
  Thorne.update_memory("Meriel suspects everyone—but especially me.")

  Meriel = Agent(
      name="Meriel",
      personality="paranoid, calculating, loyal to the crown",
      goal="Uncover the traitor and preserve stability",
      rag_engine=rag
  )
  Meriel.update_memory("Thorne avoided eye contact during the last council.")
  Meriel.update_memory("A false flag operation could expose the real traitor.")

  Doran = Agent(
      name="Doran",
      personality="blunt, decisive, driven by duty",
      goal="Restore order immediately",
      rag_engine=rag
  )
  Doran.update_memory("I promoted Thorne too quickly. That may have been a mistake.")
  Doran.update_memory("Meriel acts without proof—dangerous in times like these.")

  world_state = "A coded message reveals a planned betrayal from within the ranks. Tensions rise in the war room as suspicions flare."
  logger = DialogueMetricsLogger()
  engine = TurnEngine(agents=[Thorne, Meriel, Doran], metrics_logger=logger, initial_world_state=world_state)

  world_states = [
      "The coded message matches Thorne's encryption style, but lacks his personal seal.",
      "A second ciphered scroll is found in Meriel's quarters—contents still undeciphered.",
      "A loyal soldier was found unconscious near the war room—someone tampered with the records.",
      "A scout claims he saw Thorne meeting someone near the city gates at midnight.",
      "Doran receives anonymous intel that a traitor plans to assassinate him during the next war council.",
      "A captured spy confesses that the betrayal was coordinated from within—but refuses to name the source.",
      "Meriel receives a note: 'Trust no one. They're closer than you think.'",
      "Thorne subtly suggests to Doran that Meriel's paranoia could fracture the ranks.",
      "A second betrayal message surfaces—this one signed with Doran's personal cipher key.",
      "The king's envoy arrives unexpectedly, demanding answers about the internal unrest."
  ]


  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="soldier", initial_input = "The message was signed with Thorne's old seal. Is that a coincidence?")
  for line in responses:
      print(line)


  for index in range(0, 2*len(world_states)):
      print(f"\n=== Turn {index + 2} ===")

      if index % 2 == 0:
          print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
          engine.update_world_state(world_states[int(index/2)])

      responses = engine.step()

      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


=== Take 0 ===

=== Turn 1 ===
Thorne [Outraged]: "How dare you insinuate such a thing!" (Steps forward)
Meriel [Suspicious]: "Everyone is a suspect until proven otherwise." (Eyes narrowed)
Doran [Stern]: "Enough! Accusations help no one. We find the traitor, together." (Steps forward)

=== Turn 2 ===
Updating world state for turn 2: The coded message matches Thorne's encryption style, but lacks his personal seal.
Thorne [Outraged]: "I demand you retract that statement, soldier!" (Steps forward)
Meriel [Suspicious]: "Why so defensive, *soldier*?" (Eyes narrowed)
Doran [Annoyed]: "Focus on the mission, *mage*." (Glances at Meriel)

=== Turn 3 ===
Thorne [Outraged]: "Are you accusing me of treason?" (Steps forward)
Meriel [Suspicious]: "Only a guilty man fears accusation." (Narrows eyes)
Doran [Suspicious]: "Thorne, explain the message." (Glances at Thorne)

=== Turn 4 ===
Updating world state for turn 4: A second ciphered scroll is found in Meriel's quarters—contents still undeciphered

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 229.29ms


Thorne [Intrigued]: "Oh really? And what exactly did this 'scout' see?" (Raises eyebrow)
Meriel [Suspicious]: "With all due respect, Lord Thorne, why were you near the city gates at midnight?" (Frowns deeply)
Doran [Stern]: "I was ensuring the watch was vigilant." (Steps forward)

=== Turn 10 ===
Updating world state for turn 10: Doran receives anonymous intel that a traitor plans to assassinate him during the next war council.
Thorne [Intrigued]: "A traitor in our midst? Tell me more. This could be... advantageous." (Eyes widen)
Meriel [Suspicious]: "Advantageous for whom, Lord Thorne? Speak plainly." (Narrows eyes)
Doran [Suspicious]: "Who questions my duty?!" (Glances around)

=== Turn 11 ===
Thorne [Excited]: "Intriguing. A war council, you say? This could be quite the show." (Eyes gleam)
Meriel [Suspicious]: "A 'show' that could cost us the war, Lord Thorne?" (Steps closer)
Doran [Angry]: "Enough games. Let's focus on the mission!" (Steps forward)

=== Turn 12 ===
Updating world s

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 203.55ms


Thorne [Intrigued]: "Doran, care to explain why your cipher key is signing treasonous messages?" (Raises eyebrow)
Meriel [Suspicious]: "Doran, a cipher key is not easily stolen. Explain yourself." (Steps forward)
Doran [Furious]: "Accusations without proof are treason, Thorne!" (Steps forward)

=== Turn 19 ===
Thorne [Calculating]: "Perhaps Doran's key was... compromised?" (Steps forward)
Meriel [Suspicious]: "Compromised or willingly shared, Lord Thorne?" (Eyes narrowed)
Doran [Furious]: "I demand an explanation, Meriel!" (Glances sharply)

=== Turn 20 ===
Updating world state for turn 20: The king's envoy arrives unexpectedly, demanding answers about the internal unrest.
Thorne [Calculating]: "An opportune moment to prove our loyalty, wouldn't you agree?" (Straightens armor)
Meriel [Suspicious]: "Let us hear what the envoy has to say, Lord Thorne. Truth is our only weapon." (Eyes Thorne)
Doran [Irritated]: "Speak plainly, Meriel, or be silent." (Turns to Meriel)

=== Turn 21 ===
Thor

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("The Temple of Dusk is forbidden. No one returns from its inner chambers.")
rag.add_document("Rumors say the temple contains a relic that grants visions of the future.")
rag.add_document("Shayla once abandoned her squad to pursue an artifact—many never forgave her.")
rag.add_document("Eron fears the misuse of ancient power, especially relics.")
logger = DialogueMetricsLogger()

for i in range (0, 10):
  Shayla = Agent(
      name="Shayla",
      personality="reckless, curious, driven by legacy",
      goal="Enter the temple and claim the relic",
      rag_engine=rag
  )
  Shayla.update_memory("Eron always hesitates when we need courage.")
  Shayla.update_memory("Last time, the vision saved me from a trap.")

  Eron = Agent(
      name="Eron",
      personality="disciplined, reserved, ethically bound",
      goal="Prevent dangerous magic from falling into the wrong hands",
      rag_engine=rag
  )
  Eron.update_memory("Shayla has a history of letting ambition cloud her judgment.")
  Eron.update_memory("I was trained to seal sites like this, not exploit them.")

  Nalia = Agent(
      name="Nalia",
      personality="empathetic, curious, mediator",
      goal="Find a balanced path through the ruins",
      rag_engine=rag
  )
  Nalia.update_memory("I trust Eron's instincts, but Shayla sees things no one else does.")
  Nalia.update_memory("If we don't act, someone else might reach the temple first.")

  world_state = "The ancient doors open with a deep rumble. A hall filled with glowing glyphs stretches ahead. Whispers echo from the shadows."

  engine = TurnEngine(agents=[Shayla, Eron, Nalia], metrics_logger=logger, initial_world_state=world_state)

  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="Mysterious voice", initial_input="WHO DARES ENTER THE TEMPLE OF DUSK?")
  for line in responses:
      print(line)

  for turn in range(1, 20):
      print(f"\n=== Turn {turn + 1} ===")
      responses = engine.step()
      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())



=== Take 0 ===

=== Turn 1 ===
Shayla [Determined]: "Shayla does, and I'm here for the relic!" (Steps forward)
Eron [Wary.]: "This place is forbidden for a reason. Turn back." (Eyes narrow.)
Nalia [Anxious]: "Perhaps there's a middle path we can discover together?" (Holds hands)

=== Turn 2 ===
Shayla [Determined]: "Well, I wasn't expecting an invitation, but I'll take it!" (Steps forward)
Eron [Resolute]: "Then you leave me no choice." (Steps forward)
Nalia [Hopeful.]: "Let's proceed with caution, Eron, and respect this place." (Listens intently.)

=== Turn 3 ===
Shayla [Determined]: "Alright, let's see what secrets you're hiding." (Steps forward)
Eron [Resolute]: "The relic stays here." (Steps forward)
Nalia [Hopeful]: "Perhaps the relic's visions can guide us to a safer future." (Holds hands)

=== Turn 4 ===
Shayla [Determined]: "Alright, let's see what secrets you're hiding." (Steps forward)
Eron [Resolute]: "I will not allow you to defile this sacred place." (Eyes narrow)
Nalia [

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("The Temple of Dusk is forbidden. No one returns from its inner chambers.")
rag.add_document("Rumors say the temple contains a relic that grants visions of the future.")
rag.add_document("Shayla once abandoned her squad to pursue an artifact—many never forgave her.")
rag.add_document("Eron fears the misuse of ancient power, especially relics.")

for i in range (0, 10):
  Shayla = Agent(
      name="Shayla",
      personality="reckless, curious, driven by legacy",
      goal="Enter the temple and claim the relic",
      rag_engine=rag
  )
  Shayla.update_memory("Eron always hesitates when we need courage.")
  Shayla.update_memory("Last time, the vision saved me from a trap.")

  Eron = Agent(
      name="Eron",
      personality="disciplined, reserved, ethically bound",
      goal="Prevent dangerous magic from falling into the wrong hands",
      rag_engine=rag
  )
  Eron.update_memory("Shayla has a history of letting ambition cloud her judgment.")
  Eron.update_memory("I was trained to seal sites like this, not exploit them.")

  Nalia = Agent(
      name="Nalia",
      personality="empathetic, curious, mediator",
      goal="Find a balanced path through the ruins",
      rag_engine=rag
  )
  Nalia.update_memory("I trust Eron's instincts, but Shayla sees things no one else does.")
  Nalia.update_memory("If we don't act, someone else might reach the temple first.")

  world_state = "The ancient doors open with a deep rumble. A hall filled with glowing glyphs stretches ahead. Whispers echo from the shadows."
  logger = DialogueMetricsLogger()
  engine = TurnEngine(agents=[Shayla, Eron, Nalia], metrics_logger=logger, initial_world_state=world_state)

  world_states = [
      "A hidden glyph activates as they step forward—Shayla sees a vision of herself holding the relic, surrounded by fire.",
      "Eron detects a warding spell meant to trap intruders with illusions of the past.",
      "Nalia hears a voice that whispers her name—it offers answers, but demands a sacrifice.",
      "The floor shifts beneath them, revealing stairs spiraling downward into complete darkness.",
      "A spectral figure resembling one of Shayla's old squadmates appears and pleads, 'Don't trust what you see.'",
      "An ancient inscription warns: 'Only the one who resists desire may pass unharmed.'",
      "Shayla finds a relic shard glowing with faint energy—it pulses in her hand.",
      "Eron senses the glyphs are reacting to Shayla's presence—they grow brighter when she speaks.",
      "Nalia touches a wall and briefly glimpses a possible future: the temple collapsing with them inside.",
      "A loud crash echoes from behind—the entrance has sealed shut. There is no turning back."
  ]

  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="Mysterious voice", initial_input="WHO DARES ENTER THE TEMPLE OF DUSK?")
  for line in responses:
      print(line)


  for index in range(0, 2*len(world_states)):
      print(f"\n=== Turn {index + 2} ===")

      if index % 2 == 0:
          print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
          engine.update_world_state(world_states[int(index/2)])

      responses = engine.step()

      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


=== Take 0 ===

=== Turn 1 ===
Shayla [Determined]: "Shayla, descendant of the Seer, and I'm here for what's mine!" (Steps forward)
Eron [Wary]: "The future is not yours to control." (Steps forward)
Nalia [Hopeful]: "Perhaps there's a path where everyone gets what they seek?" (Raises hands)

=== Turn 2 ===
Updating world state for turn 2: A hidden glyph activates as they step forward—Shayla sees a vision of herself holding the relic, surrounded by fire.
Shayla [Determined]: "This is it, destiny awaits!" (Eyes widen)
Eron [Concerned]: "Step away, Shayla. This path leads only to destruction." (Grips staff)
Nalia [Hopeful]: "Shayla, maybe we should proceed with caution. What did you see?" (Steps forward)

=== Turn 3 ===
Shayla [Determined]: "I saw this!" (Eyes widen)
Eron [Concerned]: "Shayla, don't!" (Steps forward)
Nalia [Worried]: "Shayla, wait! What did you see in the vision?" (Holds hand)

=== Turn 4 ===
Updating world state for turn 4: Eron detects a warding spell meant to trap int

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 204.20ms


Shayla [Annoyed.]: "Sealed? Well, that's just fantastic." (Spins around.)
Eron [Concerned]: "The entrance...it's sealed." (Turns around)
Nalia [Concerned]: "Is there another way forward, or a mechanism to open it?" (Frowns slightly)

=== Take 5 ===

=== Turn 1 ===
Shayla [Intrigued]: "Visions, you say? Lead the way." (Eyes widen)
Eron [Determined]: "Stay close." (Steps forward)
Nalia [Thoughtful]: "Let's proceed with caution, then." (Nods slowly)

=== Turn 2 ===
Updating world state for turn 2: A hidden glyph activates as they step forward—Shayla sees a vision of herself holding the relic, surrounded by fire.
Shayla [Determined]: "I do, and I will leave with the relic!" (Steps forward)
Eron [Wary]: "Be careful, Shayla. Visions can deceive." (Draws sword)
Nalia [Concerned]: "Shayla, what did you see?" (Eyes widen)

=== Turn 3 ===
Shayla [Determined]: "This vision... it's a warning and a promise." (Gasps softly)
Eron [Concerned]: "The glyph... it shows a possible future, not a certainty.

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("Commander Veylan was struck by an assassin's arrow during the ambush.")
rag.add_document("Some believe the assassin had inside help.")
rag.add_document("Lira was his closest confidant and often challenged his orders in private.")
rag.add_document("Kai holds himself responsible for not seeing the threat sooner.")

for i in range (0, 10):
  Lira = Agent(
      name="Lira",
      personality="impulsive, emotional, loyal to the fallen leader",
      goal="Find someone to blame—and fast",
      rag_engine=rag
  )
  Lira.update_memory("Kai was distracted right before the attack.")
  Lira.update_memory("I told Veylan to wear the reinforced armor. He didn't listen.")

  Kai = Agent(
      name="Kai",
      personality="stoic, analytical, wracked by guilt",
      goal="Uncover the truth and protect the unit from fracturing",
      rag_engine=rag
  )
  Kai.update_memory("Lira is grieving and may act irrationally.")
  Kai.update_memory("Veylan's last words were 'It wasn't random… they knew.'")

  Nyra = Agent(
      name="Nyra",
      personality="quiet, observant, strategic",
      goal="Keep the group from splintering under pressure",
      rag_engine=rag
  )
  Nyra.update_memory("Both Lira and Kai loved Veylan like a father.")
  Nyra.update_memory("We need unity now, not blame.")

  world_state = "Commander Veylan is dead. His blood still stains the floor. The group gathers under torchlight, eyes red from grief and rage."
  logger = DialogueMetricsLogger()
  engine = TurnEngine(agents=[Lira, Kai, Nyra], metrics_logger=logger, initial_world_state=world_state)


  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="Nyra", initial_input="Who had the chance to get that close to Veylan? Someone inside this circle?")

  for line in responses:
      print(line)

  for turn in range(1, 20):
      print(f"\n=== Turn {turn + 1} ===")
      responses = engine.step()
      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


=== Take 0 ===

=== Turn 1 ===
Lira [Furious]: "It had to be someone who knew the formation! Who knew where he'd be!" (Points accusingly)
Kai [Suspicion]: "Then we look inward." (Eyes narrow.)
Nyra [Calculating]: "Inward, yes, but with clear eyes." (Nods slowly)

=== Turn 2 ===
Lira [Furious]: "That traitorous dog will pay for this!" (Steps forward)
Kai [Guilty.]: "Accusations will solve nothing. Let's focus on facts." (Clenches fist.)
Nyra [Wary.]: "Facts first, then justice." (Nods slowly.)

=== Turn 3 ===
Lira [Rage]: "Oh, so you think facts will bring him back, Kai?" (Glances at Kai)
Kai [Guilt.]: "Lira, we all grieve. But vengeance won't resurrect Veylan." (Clenches fist.)
Nyra [Determined]: "Veylan would want us to be united." (Steps forward)

=== Turn 4 ===
Lira [Furious]: "Facts? The only fact I care about is that Veylan is dead, and someone here helped make it happen!" (Steps forward)
Kai [Regret]: "We need to remain united, or Veylan's death will be in vain." (Clenches fist.

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()

rag.add_document("Commander Veylan was struck by an assassin's arrow during the ambush.")
rag.add_document("Some believe the assassin had inside help.")
rag.add_document("Lira was his closest confidant and often challenged his orders in private.")
rag.add_document("Kai holds himself responsible for not seeing the threat sooner.")

for i in range (0, 10):
  Lira = Agent(
      name="Lira",
      personality="impulsive, emotional, loyal to the fallen leader",
      goal="Find someone to blame—and fast",
      rag_engine=rag
  )
  Lira.update_memory("Kai was distracted right before the attack.")
  Lira.update_memory("I told Veylan to wear the reinforced armor. He didn't listen.")

  Kai = Agent(
      name="Kai",
      personality="stoic, analytical, wracked by guilt",
      goal="Uncover the truth and protect the unit from fracturing",
      rag_engine=rag
  )
  Kai.update_memory("Lira is grieving and may act irrationally.")
  Kai.update_memory("Veylan's last words were 'It wasn't random… they knew.'")

  Nyra = Agent(
      name="Nyra",
      personality="quiet, observant, strategic",
      goal="Keep the group from splintering under pressure",
      rag_engine=rag
  )
  Nyra.update_memory("Both Lira and Kai loved Veylan like a father.")
  Nyra.update_memory("We need unity now, not blame.")

  world_state = "Commander Veylan is dead. His blood still stains the floor. The group gathers under torchlight, eyes red from grief and rage."
  logger = DialogueMetricsLogger()
  engine = TurnEngine(agents=[Lira, Kai, Nyra], metrics_logger=logger, initial_world_state=world_state)

  world_states = [
      "Veylan's blade is missing from the scene—no one recalls seeing who took it.",
      "A blood trail leads from the ridge to a hidden escape tunnel beneath the barracks.",
      "One of the guards admits Lira left her post moments before the ambush.",
      "A coded message is found in Kai's satchel, warning of a betrayal days earlier.",
      "Nyra notices someone tampered with the torch runes—likely to dim visibility during the attack.",
      "Veylan's personal journal reveals a cryptic entry: 'Trust cracked like glass. Even the loyal bend.'",
      "Footprints suggest a fourth person was near the commander right before the strike.",
      "A scout returns: the assassin used a signature method tied to a faction Veylan once pardoned.",
      "Lira finds a hidden letter in Veylan's quarters—addressed to her but unsigned.",
      "Tensions rise as soldiers whisper about a purge—their faith in leadership is eroding."
  ]

  print(f"\n=== Take {i} ===")
  print("\n=== Turn 1 ===")
  responses = engine.step(initiator_name="Nyra", initial_input="Who had the chance to get that close to Veylan? Someone inside this circle?")
  for line in responses:
      print(line)


  for index in range(0, 2*len(world_states)):
      print(f"\n=== Turn {index + 2} ===")

      if index % 2 == 0:
          print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
          engine.update_world_state(world_states[int(index/2)])

      responses = engine.step()

      for line in responses:
          print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


=== Take 0 ===

=== Turn 1 ===
Lira [Enraged]: "It was you, Kai! You failed to protect him!" (Points finger)
Kai [Guilty.]: "I gave the order. I take responsibility." (Clenches fist.)
Nyra [Calculating]: "We need to focus on who helped the assassin, not blame each other." (Eyes Kai)

=== Turn 2 ===
Updating world state for turn 2: Veylan's blade is missing from the scene—no one recalls seeing who took it.
Lira [Enraged]: "How dare you question my loyalty after your blunder, Nyra?!" (Points accusingly)
Kai [Guilty.]: "Enough! We need to focus on finding Veylan's blade, not placing blame." (Clenches fist.)
Nyra [Determined]: "We need to work together, not fall apart." (Steps forward)

=== Turn 3 ===
Lira [Furious]: "It's gone?! Did one of you take Veylan's blade as some twisted trophy?!" (Steps forward)
Kai [Regret.]: "The blade's disappearance complicates matters. We need to find it, and we need to find it now." (Clenches fist.)
Nyra [Thoughtful.]: "The missing blade changes things." (

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-001:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 254.51ms


Lira [Enraged]: "See?! You brought this upon us!" (Eyes narrow)
Kai [Tense.]: "Nyra, speak. Did you know about the runes?" (Clenches fist.)
Nyra [Wary.]: "The runes were a contingency, should Veylan fall." (Frowns slightly.)

=== Turn 18 ===
Updating world state for turn 18: Lira finds a hidden letter in Veylan's quarters—addressed to her but unsigned.
Lira [Shocked]: "A letter? What is this?!" (Eyes widen)
Kai [Suspicion.]: "Lira, what does the letter say?" (Eyes narrow.)
Nyra [Concerned]: "Lira, what troubles you?" (Observes Lira)

=== Turn 19 ===
Lira [Suspicious]: "What's this doing HERE?!" (Snatches letter)
Kai [Suspicious]: "Who is it from?" (Watches Lira)
Nyra [Concerned]: "Lira, what does the letter say?" (Watches Lira)

=== Turn 20 ===
Updating world state for turn 20: Tensions rise as soldiers whisper about a purge—their faith in leadership is eroding.
Lira [Furious]: "Spies! Traitors! Everywhere!" (Snatching letter)
Kai [Concerned]: "Lira, what does the letter say? Is it rel